В 2013-2014 годах сверточные нейронные сети часто были устроены следующим образом: они принимали на вход изображение фиксированного размера, и последний сверточный слой выдавал карту признаков фиксированного размера. Затем карта признаков "вытягивалась в вектор" и обрабатывалась полносвязными слоями.

Постепенно начали предлагать и другие подходы, подволяющие обрабатывать изображения произвольного размера, например global average pooling в работе [Network In Network]($Network In Network$) или global max pooling. В текущей работе предложен еще один вариант: spatial pyramid pooling.

### Spatial Pyramid Pooling 

Сначала мы делим всю карту признаков (произвольного размера) на 4х4 части, и выполняем пулинг в каждой части. Можно использовать average pooling или max pooling. Например, если карта признаков имела размер 16х16х256, то каждая из 16 частей будет иметь размер 4х4х256, то есть каждый из 256 фильтров имеет 4х4 значения. Из этих значений выбирается максимальное. Таким образом из массива 4х4х256 мы получаем вектор длиной 256. Поскольку всего мы имели 16 частей, то получим 16 векторов длиной 256.

Затем всю карту признаков делим на 2х2 части и повторяем операцию, получая 4 вектора длиной 256. Затем ко всей карте признаков применяем max pooling и получаем еще один вектор длиной 256. Всего мы получили 16+4+1 вектор размерности 256. Выполнив их конкатенацию, мы получим вектор фиксированной размерности, который можно обработать полносвязным слоем. Размерность вектора при этом не зависит от размера исходного изображения.

<img src="assets/sppooling.jpg" width="480" align="center">

Количество уровней пулинга может быть и другим, например мы можем делить изображение сначала на 2х2, затем на 3х3, и затем на 4х4 части.

Если сторона карты признаков не делится нацело на количество частей (например, нельзя поделить карту признаков размером 13х13 на 4х4 равные части), тогда применяется следующий подход. Пусть сторона исходной карты признаков равна A, а количество частей - N. Размер окна выбирается равным $ceil(a/n)$, а шаг выбирается равным $floor(a/n)$.

### Применение в классификации изображений

В разделе 3.1.2 приводится сравнение точности сетей со spatial pyramid pooling и сетей, в которы карта признаков "вытягивается в вектор". Первые демонстрируют более высокую точность. Spatial pyramid pooling позволяет обучать одну и ту же сеть на разных размерах изображения. Авторы чередуют эпохи с обучением на изображениях размера 224х224 и 180х180, и получают еще больший прирост точности классификации.

Однако я не нашел в работе сравнения spatial pyramid pooling с такими альтернативами, как global max pooling и global average pooling. Это частные случаи spatial pyramid pooling, когда используется один уровень пулинга, и они также позволяют обучать сеть на изображениях разного размера и соотношения сторон. Поэтому из работы остается непонятным, повышает ли точность классификации наличие нескольких уровней пулинга вместо одного.

Авторы обосновывают эффективность spatial pyramid pooling тем, что он является устойчивым к пространственным деформациям, и отсылают за подробностями к работе [Beyond Bags of Features: Spatial Pyramid Matching for Recognizing Natural Scene Categories]($Beyond Bags of Features: Spatial Pyramid Matching for Recognizing Natural Scene Categories$) (2006).

>It is worth noticing that the gain of multi-level pooling is not simply due to more parameters; rather, it is because the multi-level pooling is robust to the variance in object deformations and spatial layout

Как мне кажется, это можно объяснить простыми словами: если два объекта в кадре (например, нос и хвост собаки) сместятся друг относительно друга, то вероятно так же сместятся и "образы" этих объектов на карте признаков. Если для того, чтобы преобразовать карту признаков в вектор признаков, мы используем глобальный пулинг по большому окну, тогда это смещение не повлияет на полученный вектор. Если же мы используем "вытягивание" карты признаков в вектор, то полученный вектор существенно изменится, и те же по смыслу "образы" будут обрабатывать уже другие веса полносвязного слоя.

Таким образом, переходя от "вытягивания в вектор" к глобальному пулингу, мы закладываем в архитектуру сети информацию о том, что положение "образа" объекта на карте признаков не должно влиять на результат классификации. Тогда пространственная инвариантность растет в цепочке:

*Multi-layer perceptron < CNN with flatten < CNN with global pooling*

Но это ничего не говорит о том, что несколько уровней глобального пулинга лучше одного. И наоборот, судя по тому, что spatial pyramid pooling не используется в более современных сетях, глобальный пулинг с одним уровнем все же оказался предпочтительнее.

### Применение в детектировании объектов

Авторы рассуждают о том, что вместо многократного запуска нейронной сети на разных region proposals (как в [R-CNN]($Rich feature hierarchies for accurate object detection and semantic segmentation$)), можно было бы проецировать region proposals на карту признаков, и применять глобальный пулинг по каждому из регионов в карте признаков. Таким образом, мы объединим преимущества двух сетей для детектирования: точность R-CNN и скорость [OverFeat]($OverFeat: Integrated Recognition, Localization and Detection using Convolutional Networks$). В результате на основе данной работы авторами статьи была разработана архитектура Fast R-CNN, которой посвещена отдельная [статья]($Fast R-CNN$).